# Pandas 와 Plotly를 이용한 MACD 차트 그리기

MACD 는 Moving Average Convergence Divergence 의 약자로써,  이동평균선(MA) 이 수렴과 발산을 반복한다는 원리를 이용해 단기 이동평균선과 장기이동평균선 사이의 관계를 보여주는 Gerald Appel 이 개발한 지표다. MACD에서 단기이동평균으로는 12일 장기이동평균으로는 26일을 사용하고, MACD의 9일 이동평균선을 Signal 이라고 하고, MACD와 Signal 의 차이를 Oscillator 라고 한다.

MACD: 단기이동평균(12)와 장기이동평균선(26)의 차이값
보통 주식HTS에서는 이값을 파라미터 값으로써 조정할 수 있게 되어있다. 우리나라 주식시장에 맞는 5일과 20일로 대체해서 사용하기도 한다.

Signal: MACD의 9일 이동평균값
일반적으로 9일을 설정하여 MACD의 값들을 완충 및 일반화 하는 역할을 한다.

Oscillator: MACD값과 Signal값의 차이
MACD의 값이 Signal값보다 클때는 양의 막대, 작을 때는 음의 막대가 그려진다.

MACD를 이용한 주식분석 참고 : 앤트하우스

- MACD선이 0 값을 상향돌파 할 경우 : 아래의 그림처럼 MACD선이 0을 상향돌파 할 경우는 단기이동평균선이 장기이동평균선 위에 있는 정배열을 의미한다. -> 매수 포지션( 상승 추세전환)

- MACD선이 0 값을 하향돌파 할 경우 :아래의 그림처럼 MACD선이 0을 하향돌파 할 경우는 단기 이동평균선이 장기이동평균선 아래에 있는 역배열을 의미한다 -> 매도 포지션(하락추세 전환)

- MACD 선이 Signal선을 상향돌파 할 경우 : Oscillator가 (+)로 전환된다 -> 매수 포지션

- MACD 선이 Signal선을 하향돌파 할 경우 : Oscillator가 (-)로 전환된다. → 매도 포지션

In [2]:
import pandas as pd

code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]
# pandas_read_html() 은 HTML에서 <table>></table> 태그를 찾아 자동으로 DataFrame 형식으로 만들어준다.

code_df.종목코드 = code_df.종목코드.map('{:06d}'.format)

code_df = code_df[['회사명', '종목코드']]

code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'}) 
code_df.head()


,name,code
0,GS글로벌,001250
1,HSD엔진,082740
2,KG케미칼,001390
3,LG이노텍,011070
4,OCI,010060


In [3]:
# 종목 이름을 입력하면 종목에 해당하는 코드를 불러와
# 네이버 금융("http://finance.naver.com") 에 넣어줌

def get_url(item_name, code_df):

    code = code_df.query("name=='{}'".format(item_name))['code'].to_string(index=False) 
    url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code) 
    print("요청 URL ={}".format(url)) 
    return url


item_name='신라젠'
url = get_url(item_name, code_df)

# 일자 데이터를 담은 df라는 DataFrame 정의
df = pd.DataFrame()

for page in range(1,21) :
    pg_url = '{url}&page={page}'.format(url=url, page=page) 
    df = df.append(pd.read_html(pg_url, header=0)[0], ignore_index=True) 

    
df = df.dropna()

df.head()

요청 URL =http://finance.naver.com/item/sise_day.nhn?code=215600


,날짜,종가,전일비,시가,고가,저가,거래량
1,2019.06.21,56800.0,900.0,58000.0,58300.0,56500.0,332732.0
2,2019.06.20,57700.0,1700.0,56100.0,57900.0,56100.0,487092.0
3,2019.06.19,56000.0,600.0,55900.0,56400.0,55200.0,331231.0
4,2019.06.18,55400.0,1300.0,57000.0,57400.0,55000.0,495082.0
5,2019.06.17,56700.0,100.0,57500.0,58500.0,56600.0,421203.0


In [4]:
# 한글로 된 컬럼명을 영어로 바꿔줌 
df = df.rename(columns= {'날짜': 'date', '종가': 'close', '전일비': 'diff', '시가': 'open', '고가': 'high', '저가': 'low', '거래량': 'volume'}) 
# 데이터의 타입을 int형으로 바꿔줌 
df[['close', 'diff', 'open', 'high', 'low', 'volume']] \
= df[['close', 'diff', 'open', 'high', 'low', 'volume']].astype(int) 
# 컬럼명 'date'의 타입을 date로 바꿔줌 
df['date'] = pd.to_datetime(df['date']) 
# 일자(date)를 기준으로 오름차순 정렬 
df = df.sort_values(by=['date'], ascending=True) 
# 상위 5개 데이터 확인 
df.head()


,date,close,diff,open,high,low,volume
298,2018-08-27,66300,1300,66200,67500,64700,1302620
297,2018-08-28,67400,1100,67300,69100,66500,1677504
296,2018-08-29,66100,1300,67900,68100,65300,1044419
295,2018-08-30,68500,2400,65800,68600,65400,1002923
294,2018-08-31,76400,7900,67900,79100,67700,5987954


In [11]:
def get_macd(df, short=12, long=26, t=9) :
    
    # 입력받은 값이 dataframe 이라는 것을 정의해줌
    
    df = pd.DataFrame(df)
    
    # MACD 관력 수식
    ma_12 = df.close.ewm(span=12).mean() #EMA(지수이동평균)
    ma_26 = df.close.ewm(span=26).mean()
    
    macd = ma_12 - ma_26 # MACD
    
    macds = macd.ewm(span=9).mean()
    
    macdo = macd - macds 
    
    df = df.assign(macd=macd, macds=macds, macdo=macdo).dropna()
    
    return df

In [13]:
df = get_macd(df)

df.head()

,date,close,diff,open,high,low,volume,macd,macdo,macds
298,2018-08-27,66300,1300,66200,67500,64700,1302620,0.000000,0.000000,0.000000
297,2018-08-28,67400,1100,67300,69100,66500,1677504,24.679487,10.968661,13.710826
296,2018-08-29,66100,1300,67900,68100,65300,1044419,-8.866357,-13.324239,4.457882
295,2018-08-30,68500,2400,65800,68600,65400,1002923,62.744713,38.541970,24.202744
294,2018-08-31,76400,7900,67900,79100,67700,5987954,414.695345,274.329881,140.365464


In [14]:
# plotly 를 이용해 MACD 차트 그리기

In [16]:
from plotly import tools
import plotly.offline as offline
import plotly.graph_objs as go

macd = go.Scatter(
    x=df.date,
    y=df['macd'],
    name = "MACD"
)

signal = go.Scatter(
    x=df.date,
    y=df['macds'],
    name = "Signal"
)

oscillator = go.Scatter(
    x=df.date,
    y=df['macdo'],
    name = "oscillator"
)

trade_volume = go.Scatter(
    x=df.date,
    y=df['volume'],
    name = "volume"
)

data = [macd,signal,oscillator]


# data = [celltrion] 
layout = go.Layout(title='{} MACD 그래프'.format(item_name)) 
fig = tools.make_subplots(rows=2, cols=1, shared_xaxes=True) 

for trace in data: 
    fig.append_trace(trace, 1,1) 
    
fig.append_trace(trade_volume, 2,1) 
# fig = go.Figure(data=data, layout=layout) 

offline.iplot(fig)


This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]

